In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle

class VBallManager:
    def BallTriangulation(self, cap, length = None, YellowThresh = (0.77,50), BackgroundFrames = 50, minContourArea = 20, UpdateMedian = True, MedianUpdateRate = 60, MovementThresh = 20, debug = False, FileName = None):
        SaveAsVideo = type(FileName) == str
        
        def GetBackground(imgs):
            Background = np.median(np.array(imgs), axis=0)
            return Background

        LastNImages = []

        for t in range(BackgroundFrames*MedianUpdateRate):
            res, frame = cap.read()
            if not res:
                return "lenth of vid < background frames * MedianUpdateRate ):<"
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            LastNImages.append(frame)
        Background = GetBackground(LastNImages[::MedianUpdateRate])

        if SaveAsVideo:
            video=cv2.VideoWriter(FileName,
                                  cv2.VideoWriter_fourcc(*'mp4v'), 
                                  cap.get(cv2.CAP_PROP_FPS),
                                  np.array(LastNImages[-1].shape)[[1,0]])

        centers = []
        for t in range(BackgroundFrames*MedianUpdateRate):
            print(f"Frame: {t}", end="\r")

            MovementMask = np.any((np.abs(LastNImages[t] - Background) >= MovementThresh), axis=2)

            ColorMask_ = (0.5*LastNImages[t][:, :, 0] + 0.5*LastNImages[t][:, :, 1] - LastNImages[t][:, :, 2])
            ColorMask = ((ColorMask_-np.min(ColorMask_))/(np.max(ColorMask_)-np.min(ColorMask_))>YellowThresh[0]).astype(np.uint8)

            mask = np.all([MovementMask, ColorMask], axis = 0).astype(np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25)))

            if np.count_nonzero(mask) == 0:
                video.write(cv2.cvtColor((LastNImages[t]/2).astype(np.uint8), cv2.COLOR_RGB2BGR))
                continue
            
            contour, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            if np.max(ColorMask_[mask.astype(np.bool_)]) < YellowThresh[1]:
                video.write(cv2.cvtColor((LastNImages[t]/2).astype(np.uint8), cv2.COLOR_RGB2BGR))
                continue
            
            contour = max(contour, key=cv2.contourArea)[:, 0]
            if cv2.contourArea(contour) < minContourArea:
                video.write(cv2.cvtColor((LastNImages[t]/2).astype(np.uint8), cv2.COLOR_RGB2BGR))
                continue

            x, y, w, h = cv2.boundingRect(contour)
            center = (int(x+w/2), int(y+h/2))

            if SaveAsVideo:
                video.write(cv2.circle(cv2.cvtColor((LastNImages[t]/2).astype(np.uint8), cv2.COLOR_RGB2BGR), center, 5, (0, 0, 255), -1))
            
            if debug:
                print("\n\n\n")
                plt.imshow(LastNImages[t])
                plt.scatter(*center, color="red", s=1)
                plt.show()
                # plt.imshow(Background/255)
                # plt.show()
                # plt.imshow(MovementMask1)
                # plt.show()
                # plt.imshow(ColorMask)
                # plt.show()
                plt.imshow(mask)
                plt.scatter(*center, color="red", s=1)
                plt.show()

        if not UpdateMedian: 
            del(LastNImages)
        else: 
            LastNImages = LastNImages[::MedianUpdateRate]

        while True:
            print(f"Frame: {t}", end="\r")

            if length: 
                if t >= length: break
            t += 1

            res, frame = cap.read()
            if not res:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            if UpdateMedian:
                if t % MedianUpdateRate == 0:
                    LastNImages = LastNImages[1:] + [frame]
                    Background = GetBackground(LastNImages)

            MovementMask = np.sum((np.abs(frame - Background) >= MovementThresh), axis=2).astype(np.bool_)

            ColorMask_ = (0.5*frame[:, :, 0] + 0.5*frame[:, :, 1] - frame[:, :, 2])
            ColorMask = ((ColorMask_-np.min(ColorMask_))/(np.max(ColorMask_)-np.min(ColorMask_))>YellowThresh[0]).astype(np.uint8)

            mask = np.all([MovementMask, ColorMask], axis = 0).astype(np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25)))

            if np.count_nonzero(mask) == 0:
                video.write(cv2.cvtColor((frame/2).astype(np.uint8), cv2.COLOR_RGB2BGR))
                continue
            
            if np.max(ColorMask_[mask.astype(np.bool_)]) < YellowThresh[1]:
                video.write(cv2.cvtColor((frame/2).astype(np.uint8), cv2.COLOR_RGB2BGR))
                continue

            contour, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            contour = max(contour, key=cv2.contourArea)[:, 0]
            if cv2.contourArea(contour) < minContourArea:
                video.write(cv2.cvtColor((frame/2).astype(np.uint8), cv2.COLOR_RGB2BGR))
                continue

            x, y, w, h = cv2.boundingRect(contour)
            center = (int(x+w/2), int(y+h/2))
            centers.append(center)

            if SaveAsVideo:
                video.write(cv2.circle(cv2.cvtColor((frame/2).astype(np.uint8), cv2.COLOR_RGB2BGR), center, 5, (0, 0, 255), -1))

            if debug:
                plt.imshow(frame)
                plt.scatter(*center, color="red", s=1)
                plt.show()
                # plt.imshow(Background/255, aspect='auto')
                # plt.show()
                # plt.imshow(MovementMask)
                # plt.show()
                # plt.imshow(ColorMask)
                # plt.show()
                plt.imshow(mask)
                plt.scatter(*center, color="red", s=1)
                plt.show()
        centers = np.array(centers)
        
        if SaveAsVideo:
            video.release()

            return centers, video

        return centers
    
    def ProcessVideo(self, vid, StartFrame, StorageFile=False, VidStorageFile=False, length=False):
        cap = cv2.VideoCapture(vid)
        cap.set(cv2.CAP_PROP_POS_FRAMES, StartFrame-1)
        
        result = self.BallTriangulation(cap, length=length, FileName=VidStorageFile)

        if isinstance(result, tuple):
            centers, _ = result
        else:
            centers = result

        if StorageFile:
            file = open(StorageFile,"wb")
            pickle.dump(centers, file)
            file.close()

In [ ]:

def main():
    video_path = "volleyball_clip.mp4"
    start_frame = 0  # Start from the first frame
    storage_file = "ball_tracking_data.pkl"  # File to save tracking data
    output_video = "tracked_volleyball.mp4"  # Video with tracking visualization
    
    vbm = VBallManager()
    
    print("Processing video...")
    centers = vbm.ProcessVideo(
        vid=video_path,
        StartFrame=start_frame,
        StorageFile=storage_file,
        VidStorageFile=output_video,
        length=500,  # Process full video,
    )
    
    print("Processing complete.")
    
    # Load tracking data and plot
    if centers is not None and len(centers) > 0:
        centers = np.array(centers)
        plt.figure(figsize=(10, 5))
        plt.plot(centers[:, 0], centers[:, 1], "ro-")
        plt.gca().invert_yaxis()  # Flip y-axis to match image coordinates
        plt.title("Volleyball Trajectory")
        plt.xlabel("X Position")
        plt.ylabel("Y Position")
        plt.show()
    else:
        print("No ball trajectory detected.")

if __name__ == "__main__":
    main()


In [69]:
import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture("volleyball_clip.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Video Writer for output
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Step 1: Capture initial frames for background model
background_frames = 50
frame_samples = []

for _ in range(background_frames):
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_samples.append(frame)

# Step 2: Compute background model (median)
background = np.median(np.array(frame_samples), axis=0).astype(np.uint8)

# try with MOG2
subtractor = cv2.createBackgroundSubtractorMOG2()


# Step 3: Process frames to detect the ball
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Step 4: Compute Movement Mask (detect changes from background)
    # movement_mask = np.any(np.abs(frame_rgb - background) > 20, axis=2).astype(np.uint8) * 255  # Convert to 255 for display
    
    movement_mask = subtractor.apply(frame_rgb)

    # Step 5: Compute Color Mask (detect yellow)
    # color_mask_value = 0.5 * frame_rgb[:, :, 0] + 0.5 * frame_rgb[:, :, 1] - frame_rgb[:, :, 2]
    # color_mask = ((color_mask_value - np.min(color_mask_value)) / (np.max(color_mask_value) - np.min(color_mask_value)) > 0.77).astype(np.uint8) * 255
    
    # Use HSV color space to detect yellow
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    lower_yellow = np.array([90, 150, 90])  # Lower bound for hue, saturation, and value
    upper_yellow = np.array([100, 255, 255])  # Upper bound
    color_mask = cv2.inRange(frame_hsv, lower_yellow, upper_yellow)


    # Step 6: Combine both masks
    mask = (movement_mask & color_mask).astype(np.uint8) * 255

    # Step 7: Apply Morphological Operations
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25)))

    # Step 8: Find Contours (detect the volleyball)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Select the largest contour (assuming it's the ball)
        largest_contour = max(contours, key=cv2.contourArea)

        # Get bounding box and center of the detected ball
        x, y, w, h = cv2.boundingRect(largest_contour)
        center = (int(x + w / 2), int(y + h / 2))

        # Draw the detected ball on the frame
        cv2.circle(frame, center, 10, (0, 0, 255), -1)

    # Step 9: Display masks for debugging
    cv2.imshow("Movement Mask", movement_mask)  # Movement detection
    cv2.imshow("Color Mask", color_mask)  # Yellow detection
    cv2.imshow("Final Mask (Movement & Color)", mask)  # Combined mask
    cv2.imshow("Original", frame)  # Combined mask

    # Step 10: Write frame to output video
    out.write(frame)

    # Press 'q' to exit debugging mode
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
